In [ ]:
from tqdm import tqdm

In [ ]:
!pip install torchvision

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.utils import save_image

In [ ]:
model = models.vgg19(pretrained=True).features

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:05<00:00, 96.8MB/s]


In [ ]:
print(model)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU(inplace=True)
  (18): MaxPoo

In [ ]:
class VGG(nn.Module):
  def __init__(self):
    super(VGG, self).__init__()

    self.chosen_features = ['0', '5', '10', '19', '28']
    self.model = models.vgg19(pretrained=True).features[:29]

  def forward(self, x):
    features = [] # relevant list of features for the chosen_features

    for layer_num, layer in enumerate(self.model):
      x = layer(x)

      if str(layer_num) in self.chosen_features:
        features.append(x)

    return features

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available else "cpu")
image_size = 356

In [ ]:
def load_image(image_name):
  image = Image.open(image_name)
  image = loader(image).unsqueeze(0) # we need to add the additional dimension for the batch size
  return image.to(device)

In [ ]:
# Read image
img = Image.open("/content/drive/MyDrive/Jetapult/bubble.png")

In [ ]:
orig_width, orig_height = img.size
print("orig_width", orig_width)
print("orig_height", orig_height)

In [ ]:
orig_width, orig_height = 356, 356

In [ ]:
loader = transforms.Compose(
    [
        transforms.Resize((orig_width, orig_height)),
        transforms.ToTensor(),
        # transforms.Normalize(mean=[], std=[])
    ]
)

In [ ]:
original_img = load_image("/content/drive/MyDrive/Jetapult/trc-killers-flower-moon-drum.jpg")
style_img = load_image("/content/drive/MyDrive/Jetapult/Warli-painting-Chitya-Laxman-05.jpg")

In [ ]:
model = VGG().to(device).eval()

In [ ]:
# generated = torch.rand(original_img.shape, device=device, requires_grad=True)
generated = original_img.clone().requires_grad_(True)

In [ ]:
#Hyperparameters
total_steps = 6000
learning_rate = 0.001
alpha = 1
beta = 0.01
optimizer = optim.Adam([generated], lr=learning_rate)

In [ ]:
for step in tqdm(range(total_steps)):
  #for how many times the image is going to be modified
  generated_features = model(generated)
  original_img_features = model(original_img)
  style_features = model(style_img)

  style_loss = original_loss  = 0

  for gen_feature, orig_feature, style_feature in zip(
      generated_features, original_img_features, style_features
  ):
    batch_size, channel, height, width = gen_feature.shape # we have to consider for every block
    original_loss += torch.mean((gen_feature - orig_feature) ** 2)

    # Compute Gram Matrix
    # batch_size if 1 as we are sending only one image
    # here we are multiplying every pixel value from each channel to every other channel for the
    # generated features and we will end up having shape channel by channel and this is then later on
    # going to be subtracted by style gram matrix

    # you can view it as the GRAM MATRIX is calculating some sort of
    # correlation matrix, if the pixel colors are similar across the channels
    # of the genrated image and style image then that sort of results
    # in two pictures having similar style
    G = gen_feature.view(channel, height*width).mm(
        gen_feature.view(channel, height*width).t()
    )

    A = style_feature.view(channel, height*width).mm(
        style_feature.view(channel, height*width).t()
    )

    style_loss += torch.mean((G-A)**2)

  total_loss = alpha*original_loss + beta*style_loss
  optimizer.zero_grad()
  total_loss.backward()
  optimizer.step()

  if step%200==0:
    print(total_loss)
    save_image(generated, "/content/drive/MyDrive/Jetapult/generated/"+"generated_warli"+str(step)+".png")




  0%|          | 0/6000 [00:00<?, ?it/s]

tensor(871047.7500, device='cuda:0', grad_fn=<AddBackward0>)


  3%|▎         | 203/6000 [00:37<16:02,  6.02it/s]

tensor(46933.4219, device='cuda:0', grad_fn=<AddBackward0>)


  7%|▋         | 400/6000 [01:12<16:53,  5.52it/s]

...

 93%|█████████▎| 5603/6000 [16:58<01:07,  5.89it/s]

tensor(1101.5577, device='cuda:0', grad_fn=<AddBackward0>)


 97%|█████████▋| 5803/6000 [17:34<00:33,  5.84it/s]

tensor(1042.9532, device='cuda:0', grad_fn=<AddBackward0>)


100%|██████████| 6000/6000 [18:10<00:00,  5.50it/s]
